In [1]:
import Pkg; Pkg.activate("..")

  Activating project at `~/com.github/lucifer1004/AdventOfCode.jl`


In [2]:
using AdventOfCode

In [3]:
year = 2015
day = 19

19

In [4]:
input = get_input(year, day)

"Al => ThF\nAl => ThRnFAr\nB => BCa\nB => TiB\nB => TiRnFAr\nCa => CaCa\nCa => PB\nCa => PRnFAr\nCa => SiRnFYFAr\nCa => SiRnMgAr\nCa => SiTh\nF => CaF\nF => PMg\nF => SiAl\nH => CRnAlAr\nH => CRnFYFYFAr\nH => CRnFYMgAr\nH => CRnMgYFAr\nH => HCa\nH => NRnFYFAr\nH => NRnMgAr\nH => NTh\nH => OB\nH" ⋯ 426 bytes ⋯ "PBFYCaCaFArCaCaCaCaSiThCaSiRnPRnFArPBSiThPRnFArSiRnMgArCaFYFArCaSiRnSiAlArTiTiTiTiTiTiTiRnPMgArPTiTiTiBSiRnSiAlArTiTiRnPMgArCaFYBPBPTiRnSiRnMgArSiThCaFArCaSiThFArPRnFArCaSiRnTiBSiThSiRnSiAlYCaFArPRnFArSiThCaFArCaCaSiThCaCaCaSiRnPRnCaFArFYPMgArCaPBCaPBSiRnFYPBCaFArCaSiAl"

In [5]:
parsed_input = parse_input(input)

45-element Vector{SubString{String}}:
 "Al => ThF"
 "Al => ThRnFAr"
 "B => BCa"
 "B => TiB"
 "B => TiRnFAr"
 "Ca => CaCa"
 "Ca => PB"
 "Ca => PRnFAr"
 "Ca => SiRnFYFAr"
 "Ca => SiRnMgAr"
 "Ca => SiTh"
 "F => CaF"
 "F => PMg"
 ⋮
 "P => CaP"
 "P => PTi"
 "P => SiRnFAr"
 "Si => CaSi"
 "Th => ThCa"
 "Ti => BP"
 "Ti => TiTi"
 "e => HF"
 "e => NAl"
 "e => OMg"
 ""
 "CRnSiRnCaPTiMgYCaPTiRnFArSiThFA" ⋯ 427 bytes ⋯ "gArCaPBCaPBSiRnFYPBCaFArCaSiAl"

In [6]:
sample = """e => H
e => O
H => HO
H => OH
O => HH

HOH"""

"e => H\ne => O\nH => HO\nH => OH\nO => HH\n\nHOH"

In [7]:
parsed_sample = parse_input(sample)

7-element Vector{SubString{String}}:
 "e => H"
 "e => O"
 "H => HO"
 "H => OH"
 "O => HH"
 ""
 "HOH"

In [8]:
function preprocess(input)
    lastline = input[end]
    source = [lastline[idx] for idx in findall(r"[A-Z][a-z]?", lastline)]
    ruledict = DefaultDict{String, Vector{Vector{String}}}(() -> Vector{String}[])
    foreach(input[1:end-2]) do line
        from, to = split(line, " => ")
        to = [to[idx] for idx in findall(r"[A-Z][a-z]?", to)]
        push!(ruledict[from], to)
    end

    elements = Set{String}()
    for el in source
        push!(elements, el)
    end
    for (from, tos) in ruledict
        push!(elements, from)
        for to in tos
            for el in to
                push!(elements, el)
            end
        end
    end

    mapping = Dict(el => 'a' + (i - 1) for (i, el) in enumerate(elements))
    source_mapped = join(mapping[el] for el in source)
    ruledict_mapped = Dict{Char, Vector{String}}(
        mapping[from] => [join(mapping[el] for el in to) for to in tos] for (from, tos) in ruledict
    )

    return ruledict_mapped, source_mapped, mapping
end

preprocess (generic function with 1 method)

In [9]:
function part_one(input)
    ruledict, source, _ = preprocess(input)
    results = Set{String}()
    for (idx, atom) in enumerate(collect(source))
        for to in get(ruledict, atom, [])
            new_source = source[1:idx-1] * to * source[idx+1:end]
            push!(results, new_source)
        end
    end

    return length(results)
end

part_one (generic function with 1 method)

In [10]:
part_one(parsed_sample)

4

In [11]:
part_one_ans = part_one(parsed_input)

518

In [12]:
submit_answer(year, day, part_one_ans)

"<!DOCTYPE html>\n<html lang=\"en-us\">\n<head>\n<meta charset=\"utf-8\"/>\n<title>Day 19 - Advent of Code 2015</title>\n<!--[if lt IE 9]><script src=\"/static/html5.js\"></script><![endif]-->\n<link href='//fonts.googleapis.com/css?family=Source+Code+Pro:300&subset=latin,latin-ext'" ⋯ 2838 bytes ⋯ "Name(o)[0];a.async=1;a.src=g;m.parentNode.insertBefore(a,m)\n})(window,document,'script','//www.google-analytics.com/analytics.js','ga');\nga('create', 'UA-69522494-1', 'auto');\nga('set', 'anonymizeIp', true);\nga('send', 'pageview');\n</script>\n<!-- /ga -->\n</body>\n</html>"

In [13]:
"""
The solution to the second part is based on the following observation:

All rules can be categorized into four types:

- α⟹βγ
- α⟹βRnγAr
- α⟹βRnγYδAr
- α⟹βRnγYδYεAr

Importantly, α, β, γ, δ and ε cannot be Rn, Ar or Y. So the second part reduces to a simple counting:

- Each pair of Rn and Ar implies that we can save 2 operations.
- Each Y implies we can save 2 operations.
- We start from e.
- So the number of operations equals to: length(source) - 2 * number of Rn-Ar pairs - 2 * number of Ys - 1 (e)
"""
function part_two(input)
    _, source, mapping = preprocess(input)
    lch = mapping["Rn"]
    rch = mapping["Ar"]
    mch = mapping["Y"]
    L = count((==)(lch), source)
    R = count((==)(rch), source)
    M = count((==)(mch), source)
    return length(source) - L - R - M * 2 - 1
end

part_two

In [14]:
part_two_ans = part_two(parsed_input)

200

In [15]:
submit_answer(year, day, part_two_ans, 2)

"<!DOCTYPE html>\n<html lang=\"en-us\">\n<head>\n<meta charset=\"utf-8\"/>\n<title>Day 19 - Advent of Code 2015</title>\n<!--[if lt IE 9]><script src=\"/static/html5.js\"></script><![endif]-->\n<link href='//fonts.googleapis.com/css?family=Source+Code+Pro:300&subset=latin,latin-ext'" ⋯ 2843 bytes ⋯ "Name(o)[0];a.async=1;a.src=g;m.parentNode.insertBefore(a,m)\n})(window,document,'script','//www.google-analytics.com/analytics.js','ga');\nga('create', 'UA-69522494-1', 'auto');\nga('set', 'anonymizeIp', true);\nga('send', 'pageview');\n</script>\n<!-- /ga -->\n</body>\n</html>"